In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import requests
import math

In [ ]:
def get_fuel_stations_near_point(lat, lon, radius, api_key):
    # Define search query for fuel stations near a specific point
    query = "fuel stations"
    
    # Send request to Google Places API with location and radius
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={query}&location={lat},{lon}&radius={radius}&key={api_key}"
    response = requests.get(url)
    data = response.json()
    
    # Extract locations
    locations = []
    if 'results' in data:
        for result in data['results']:
            name = result['name']
            lat = result['geometry']['location']['lat']
            lon = result['geometry']['location']['lng']
            locations.append({'Name': name, 'Latitude': lat, 'Longitude': lon})
    
    return locations

In [ ]:
def create_search_radius(centroid, distance_km):
    # Approximate radius in meters
    radius_meters = distance_km * 1000
    return radius_meters

In [ ]:
# Load the centroid shapefile (replace with your own shapefile path)
shapefile_path = "path_to_your_centroid_shapefile.shp"
centroids = gpd.read_file(shapefile_path)

In [ ]:
# Initialize an empty list to collect all fuel stations
all_fuel_stations = []

# Your Google Maps API key
api_key = 'YOUR_API_KEY'

# Define the search radius in kilometers
search_radius_km = 10


In [ ]:
# Iterate over each centroid
for _, row in centroids.iterrows():
    centroid_geom = row['geometry']
    lat, lon = centroid_geom.y, centroid_geom.x
    
    # Define the search radius in meters
    radius = create_search_radius(centroid_geom, search_radius_km)
    
    # Get fuel stations within the radius from the centroid
    fuel_stations_nearby = get_fuel_stations_near_point(lat, lon, radius, api_key)
    
    # Add to the overall list
    all_fuel_stations.extend(fuel_stations_nearby)

In [ ]:
# Convert the final list of stations into a GeoDataFrame
latitudes = [station['Latitude'] for station in all_fuel_stations]
longitudes = [station['Longitude'] for station in all_fuel_stations]
points = [Point(lon, lat) for lon, lat in zip(longitudes, latitudes)]
gdf = gpd.GeoDataFrame(all_fuel_stations, geometry=points)

In [ ]:
# Define output GeoPackage path
output_gpkg = "lithuania_fuel.gpkg"

# Save as GeoPackage
gdf.to_file(output_gpkg, layer='fuel_stations', driver="GPKG")

print(f"GeoPackage saved as {output_gpkg} with {len(gdf)} fuel stations.")
